### Задачи исследования

#### 1. Осмотр данных
- посмотрим начало и конец датасета
- сделаем предварительные выводы
- поищем аномалии


#### 2. Обработка данных
- найти и обработать пропуски
- заменить некорректные типы данных
- удалить дубликаты
- выделить леммы


#### 3. Ответы на вопросы
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?


#### 4. Результаты исследования
- Общие выводы
- Рекомендации

### Этап 1. Получение данных, импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import pprint

from collections import Counter

In [2]:
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
df.tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-info">

**Данные выглядят некорректно:** 
    
- отрицательные значения в детях, в днях стажа 
- отсутствуют строки в днях стажа и доходе 
- некачественные названия столбцов 
- некорректные типы данных 
- в днях стажа и в доходе плавающая точка 
- неудобные данные для анализа 
- дни стажа лучше перевести в года 
- прыгающий регистр букв в образовании 
- есть дублирующие признаки — избыточны образование и его идентификатор и такая же проблема у семейного положения 
- непонятна валюта месячного дохода - минимальное значение в возрасте — 0
   
    
</div>

### Этап 2. Обработка данных

#### Поищем дубликаты, приведем названия столбцов в порядок, поработаем с признаками.

In [7]:
df.columns.tolist()

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

Переименуем столбцы

In [8]:
cols = ['number_of_children',
        'days_employed',
        'client_age_in_years',
        'education_level',
        'education_level_id',
        'family_status',
        'family_status_id',
        'gender',
        'employment_type',
        'credit_repayment_status',
        'monthly_income',
        'credit_purpose'
       ]
df.set_axis(cols, axis='columns', inplace=True)

In [9]:
df.columns.tolist()

['number_of_children',
 'days_employed',
 'client_age_in_years',
 'education_level',
 'education_level_id',
 'family_status',
 'family_status_id',
 'gender',
 'employment_type',
 'credit_repayment_status',
 'monthly_income',
 'credit_purpose']

#### Подсчитаем суммарное количество пропусков

In [10]:
df.isnull().sum().sort_values()

number_of_children            0
client_age_in_years           0
education_level               0
education_level_id            0
family_status                 0
family_status_id              0
gender                        0
employment_type               0
credit_repayment_status       0
credit_purpose                0
days_employed              2174
monthly_income             2174
dtype: int64

<div class="alert alert-info">

Одинаковое количество пропусков в днях работы и доходе в месяц. Возможно, эти люди никогда не работали, так как молоды или работали не официально. Ну либо данные некорректно выгружены. Позже посмотрим на них подробнее.
    
</div>

#### Заменим пропуски на нули. Подставлять  медианные значения некорректно, так как это повысит шанс получения кредита, но не повысит качество анализа

In [11]:
df['days_employed'] = df['days_employed'].fillna(0)

In [12]:
df['monthly_income'] = df['monthly_income'].fillna(0)

In [13]:
df.isnull().sum()

number_of_children         0
days_employed              0
client_age_in_years        0
education_level            0
education_level_id         0
family_status              0
family_status_id           0
gender                     0
employment_type            0
credit_repayment_status    0
monthly_income             0
credit_purpose             0
dtype: int64

#### Проверим дубликаты

In [14]:
df.duplicated().sum()

54

<div class="alert alert-info">

54 дубля, скорее всего из-за ошибки правил валидации на сервере, заявки должны проверяться на дубли сначала там 
    
</div>

#### Удалим дубли и сбросим индекс

In [15]:
df = df.drop_duplicates().reset_index(drop=True)

In [16]:
df.duplicated().sum()

0

In [17]:
df.head(5)

,number_of_children,days_employed,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Исследуем каждый столбец в отельности

In [18]:
df['number_of_children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

#### Значения -1 и 20 выглядят странно. Подсчитаем количество таких строк

In [19]:
df[df['number_of_children'] == -1]['number_of_children'].count()

47

#### Заменим -1 на 1

In [20]:
df['number_of_children'] = df['number_of_children'].replace(-1, 1)

In [21]:
df[df['number_of_children'] == -1]['number_of_children'].count()

0

#### Подсчитаем количество строк с 20 детьми

In [22]:
df[df['number_of_children'] == 20]['number_of_children'].count()

76

#### Cделаем сортировку, чтобы убедиться в корректности данных

In [23]:
df['number_of_children'].value_counts()

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: number_of_children, dtype: int64

#### Cчитаем ошибкой и заменяем на 2

In [24]:
df['number_of_children'] = df['number_of_children'].replace(20, 2)

In [25]:
df['number_of_children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

#### Трудовой стаж в днях

In [26]:
df['days_employed'].describe()

count     21471.000000
mean      56821.423140
std      135010.270744
min      -18388.949901
25%       -2522.536607
50%        -989.271304
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

<div class="alert alert-info">
    
Выглядят хаотично. Есть отрицательные значения, значения не в целых числах да и вообще дни не очень удобно использовать. Разберемся cначала с отрицательными значениями
   
    
</div>

In [27]:
df['days_employed'] = df['days_employed'].abs()

In [28]:
df['days_employed'].describe()

count     21471.000000
mean      60307.713617
std      133489.355354
min           0.000000
25%         620.736110
50%        1818.689386
75%        4794.911909
max      401755.400475
Name: days_employed, dtype: float64


<div class="alert alert-info">

Стало лучше, но максимальные значения выглядят аномально. 
Найдем самого великовозрастного человека в наборе.
Если предположить, что он работал с 16 лет, то:
   
    
</div>

In [29]:
max_days_of_employment = (df['client_age_in_years'].max() - 16) * 365
max_days_of_employment

21535

In [30]:
df[df['days_employed'] > max_days_of_employment]['days_employed'].count() / 21525 * 100

16.004645760743323



<div class="alert alert-info">

Много, 16% от всего набора. Переведём в года и добавим новый столбец  
   
    
</div>

In [31]:
df['years_employed'] = df['days_employed'] / 365
df['years_employed'] = df['years_employed'].astype(int)
del df['days_employed']

In [32]:
df.head()

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932


#### Разберемся с возрастом

In [33]:
df['client_age_in_years'].describe()

count    21471.000000
mean        43.279074
std         12.574291
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: client_age_in_years, dtype: float64

#### Подсчитаем людей с возрастом меньше 0, 10 и 18 лет, таких 101

In [34]:
df[df['client_age_in_years'] == 0]['client_age_in_years'].count()

101

In [35]:
df[df['years_employed'] > 70].groupby(['client_age_in_years'])['client_age_in_years'].count().sort_values()

client_age_in_years
22      1
28      1
31      1
35      1
26      2
33      2
27      3
32      3
34      3
36      3
74      4
39      4
37      5
41      6
73      6
40      7
38      8
42      9
43      9
44     10
45     11
47     13
46     13
0      17
48     20
72     28
49     30
71     48
70     54
50     61
51     73
69     74
68     80
52     95
53    105
67    132
65    136
66    139
54    145
55    162
64    179
56    184
63    192
58    208
57    212
61    214
62    235
60    243
59    254
Name: client_age_in_years, dtype: int64

#### В образовании уников 15, в ID уников 5

In [36]:
len(df['education_level'].unique())

15

In [37]:
df['education_level_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

#### Приведем строки к нижнему регистру

In [38]:
df['education_level'] = df['education_level'].str.lower()

In [39]:
df['education_level'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

#### Посмотрим на семейный статус, приведем все к нижнему регистру

In [40]:
df['family_status'] = df['family_status'].str.lower()

In [41]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

#### Посмотрим что с полом

In [42]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

####  Есть одна запись с чем-то непонятным

In [43]:
df[df['gender'] == 'XNA'].count()

number_of_children         1
client_age_in_years        1
education_level            1
education_level_id         1
family_status              1
family_status_id           1
gender                     1
employment_type            1
credit_repayment_status    1
monthly_income             1
credit_purpose             1
years_employed             1
dtype: int64

#### Статусы в порядке

In [44]:
df['employment_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

#### Смотреть на доходы с 6 знаками после запятой неудобно.  В контексте задачи точностью даходов не важна. Приведем к целочисленным значениям

In [45]:
df['monthly_income'] = df['monthly_income'].astype(int)
df.head()

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932


#### Узнаем количество уникальных значений — 38

In [46]:
unique_credit_purposes = df['credit_purpose'].unique().tolist()
unique_credit_purposes

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

### Этап 3. Ответим на вопросы

### Зависимость между наличием детей и возвратом кредита в срок

In [47]:
number_of_children_set = df.groupby('number_of_children')['credit_repayment_status'].sum() / \
    df.groupby('number_of_children')['credit_repayment_status'].count() * 100
number_of_children_set

number_of_children
0    7.535266
1    9.163921
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: credit_repayment_status, dtype: float64

In [48]:
grouped_number_of_children = df.groupby('number_of_children')['credit_repayment_status'].count()
grouped_number_of_children

number_of_children
0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: credit_repayment_status, dtype: int64

In [49]:
children_pivot = df.pivot_table(index = ['number_of_children'], \
                                columns = ['credit_repayment_status'], \
                                values = 'credit_purpose', aggfunc='count')

children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
children_pivot

credit_repayment_status,0,1,ratio
number_of_children,,,
0,13044.0,1063.0,8.149341
1,4411.0,445.0,10.088415
2,1926.0,202.0,10.488058
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


<div class="alert alert-info">

Люди с детьми или без отдают кредиты примерно на одном и том же уровне. юди с 5 детьми показывают 0, но выборка по ним не репрезентативна. Бездетные отдают кредиты хуже.
  
    
</div>



### Зависимость между семейным положением и возвратом кредита в срок

In [50]:
family_status_set = df.groupby('family_status')['credit_repayment_status'].sum() / \
      df.groupby('family_status')['credit_repayment_status'].count() * 100
family_status_set

family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.320202
женат / замужем          7.542126
не женат / не замужем    9.750890
Name: credit_repayment_status, dtype: float64

In [51]:
family_status_pivot = df.pivot_table(index=['family_status'],\
                    columns=['credit_repayment_status'], values='monthly_income', aggfunc='count')
family_status_pivot['ratio'] = family_status_pivot[1] / family_status_pivot[0] * 100
family_status_pivot

credit_repayment_status,0,1,ratio
family_status,,,
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3775,388,10.278146
женат / замужем,11413,931,8.157364
не женат / не замужем,2536,274,10.804416


<div class="alert alert-info">

Семейное положение влияет. Самые низкие показатели у разведенных и вдов — можно предположить, что возвращаемость кредитов зависит в том числе от наличия кормильца или супруга в семье. 
   
    
</div>

### Зависимость между уровнем дохода и возвратом кредита в срок

In [52]:
df['monthly_income'].quantile([0.25,0.5,0.75])

0.25     88946.5
0.50    135716.0
0.75    195751.5
Name: monthly_income, dtype: float64

#### Pаспределим уровень дохода на 4 группы, присвоим статусы каждой группе

In [53]:
def income_status(monthly_income):
    if monthly_income <= 88946.5:
            return 'Низкий уровень дохода'
    if monthly_income <= 135716.0:
            return 'Средний уровень дохода'
    if monthly_income < 195751.5:
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'

In [54]:
df['income_status'] = df['monthly_income'].apply(income_status)

In [55]:
df.head()

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed,income_status
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,Сверхвысокий уровень дохода
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,Средний уровень дохода
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,Высокий уровень дохода
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,Сверхвысокий уровень дохода
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,Высокий уровень дохода


In [56]:
monthly_income_set = df.groupby('income_status')['credit_repayment_status'].sum() / \
      df.groupby('income_status')['credit_repayment_status'].count() * 100
monthly_income_set.sort_values()

income_status
Сверхвысокий уровень дохода    7.134873
Низкий уровень дохода          7.805514
Средний уровень дохода         8.476155
Высокий уровень дохода         9.018073
Name: credit_repayment_status, dtype: float64

In [57]:
monthly_income_pivot = df.pivot_table(index=['income_status'], columns=['credit_repayment_status'], values='education_level_id', aggfunc='count')
monthly_income_pivot['ratio'] = monthly_income_pivot[1] / monthly_income_pivot[0] * 100
monthly_income_pivot

credit_repayment_status,0,1,ratio
income_status,,,
Высокий уровень дохода,4883,484,9.911939
Низкий уровень дохода,4949,419,8.466357
Сверхвысокий уровень дохода,4985,383,7.683049
Средний уровень дохода,4913,455,9.261144


#### Проверим количество людей в каждой из групп

In [58]:
df['income_status'].value_counts(normalize = True)

Сверхвысокий уровень дохода    0.250012
Средний уровень дохода         0.250012
Низкий уровень дохода          0.250012
Высокий уровень дохода         0.249965
Name: income_status, dtype: float64

<div class="alert alert-info">

Зависимость возвращаемости кредитов от уровня доходов незначительна.
Хуже всего покатель у людей со сверхвысокими и низкими доходами. Если по низким доходам связь очевидна, то сверхвысокие пока не ясно. 
    
</div>

### Как разные цели кредита влияют на его возврат в срок

In [59]:
credit_purpose_keys= []
credit_purpose_list = list(df.credit_purpose.unique())
for value in range(len(credit_purpose_list)):
    if 'образован' in credit_purpose_list[value]:
        credit_purpose_keys.append('Образование')
    elif 'авто' in credit_purpose_list[value]:
        credit_purpose_keys.append('Автомобиль')
    elif 'свадь' in credit_purpose_list[value]:
        credit_purpose_keys.append('Свадьба')
    else:
        credit_purpose_keys.append('Недвижимость')
        
credit_purpose_keys

['Недвижимость',
 'Автомобиль',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Автомобиль',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Свадьба',
 'Образование',
 'Недвижимость',
 'Недвижимость',
 'Образование',
 'Автомобиль',
 'Автомобиль',
 'Образование',
 'Образование',
 'Недвижимость',
 'Автомобиль',
 'Недвижимость',
 'Образование']

In [60]:
credit_purpose_list

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

#### Сформировали словарь, теперь передатдим в map()

In [61]:
credit_purpose_dict = dict(zip(credit_purpose_list, credit_purpose_keys))

In [62]:
df['credit_purpose_status'] = df['credit_purpose'].map(credit_purpose_dict)

In [63]:
credit_purpose_pivot = df.pivot_table(index=['credit_purpose_status'], columns=['credit_repayment_status'], values='education_level_id', aggfunc='count')
credit_purpose_pivot['ratio'] = credit_purpose_pivot[1] / credit_purpose_pivot[0] * 100
credit_purpose_pivot

credit_repayment_status,0,1,ratio
credit_purpose_status,,,
Автомобиль,3905,403,10.320102
Недвижимость,10032,782,7.795056
Образование,3644,370,10.153677
Свадьба,2149,186,8.655188


<div class="alert alert-info">
Цели кредита влияют. Хуже всего выглядят свадьбы и недвижимость. Это логично, ипотека — самая крупная кредитная и рисковая сделка для частного лица. Свадьбы — это не инвестиционный проект и неликвидный кредит.

    
</div>

### Этап 4. Результаты исследования



<div class="alert alert-info">
Есть зависимости между целями кредита, семейным положением, уровнем дохода и количеством детей в семье.  Отличие мало, но 2% при больших числах могут оказать существенное влияние на кредитный портфель банка. Особое внимание необходимо уделить вдовам/вдовцам или молодоженам, у них риски по невозврату кредитов выше.
    
</div>

#### Рекомендации для банка

1) Обратить внимание на правила валидации на сервере, чтобы больше не сталкиваться с проблемой дублей.

2) Продумать сценарий, когда отличается дата или время заявки, а уникальный идентификатор пользователя один и тот же, иначе фактически дубли будут, но метод duplicated() их не найдет.

3) Считать стаж работы в годах, пользователи чаще всего не знают свой стаж в днях.

4) Показывать операторам в банке или пользователям на сайте подсказки заранее, когда вводят сверхвысокие/сверхнизкие значения в полях: возраст, стаж, количество детей.

5) Блокировать отправку заявок с заведомо некорректными значениями, которые могут быть важны для анализа, например - с отрицательным количеством детей.